In [2]:
"""
Fully specified, error-safe code block for Design 1:
"Among current contraceptive users, classify who is high-risk for discontinuation."

Assumptions:
- Input file: merged_dataset.csv
- Key columns exist as in your schema:
  CURRENT_USE_TYPE, CONTRACEPTIVE_USE_AND_INTENTION, INTENTION_USE,
  LAST_METHOD_DISCONTINUED, REASON_DISCONTINUED, plus demographic/fertility/method covariates.

This code:
1. Loads data
2. Filters to current users
3. Constructs a binary target HIGH_RISK_DISCONTINUE
4. Selects a feature set with no label leakage
5. Prepares X, y for modeling (no model is trained here, so you can plug into your existing pipeline)

You can paste this into one or more cells in your notebook and run as-is.
"""

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

# -------------------------------------------------------------------
# 1. Load data
# -------------------------------------------------------------------

DATA_PATH = "../../data/interim/merged_dataset.csv"  # adjust if needed

df = pd.read_csv(DATA_PATH)

print("Raw data shape:", df.shape)
print("Columns:", list(df.columns))

# -------------------------------------------------------------------
# 2. Helper: safe membership check on mixed code/label columns
# -------------------------------------------------------------------

def is_in(series: pd.Series, values) -> pd.Series:
    """
    Robust membership check that works if the column has mixed types
    (e.g. numeric codes and string labels).
    """
    values_str = set(str(v) for v in values)
    return series.astype(str).isin(values_str)


# -------------------------------------------------------------------
# 3. Filter to CURRENT users only
#    We treat '3' and 'Current user' as current user status.
# -------------------------------------------------------------------

if "CURRENT_USE_TYPE" not in df.columns:
    raise ValueError("Column 'CURRENT_USE_TYPE' not found in the dataset.")

current_user_values = ["3", "Current user"]

df_current = df[is_in(df["CURRENT_USE_TYPE"], current_user_values)].copy()
print("Filtered to current users. Shape:", df_current.shape)

# -------------------------------------------------------------------
# 4. Construct HIGH_RISK_DISCONTINUE target (Design 1)
#    Logic:
#    - High risk (1) if:
#        * current user AND
#        * (intention suggests 'using but intends to stop' OR negative history)
#    - Low risk (0) if:
#        * current user AND
#        * intention suggests 'using and intends to continue'
#    - Others: target = NaN (dropped later)
# -------------------------------------------------------------------

# 4.1 Intention-based risk using CONTRACEPTIVE_USE_AND_INTENTION
high_risk_intention_values = [
    "3",                           # numeric code for "Using but intends to stop" (from your description)
    "Using but intends to stop",   # label form
]

low_risk_intention_values = [
    "1",                           # numeric code for "Using and intends to continue"
    "Using and intends to continue"
]

if "CONTRACEPTIVE_USE_AND_INTENTION" in df_current.columns:
    intention_high_risk = is_in(df_current["CONTRACEPTIVE_USE_AND_INTENTION"],
                                high_risk_intention_values)
    intention_low_risk = is_in(df_current["CONTRACEPTIVE_USE_AND_INTENTION"],
                               low_risk_intention_values)
else:
    # If column is missing (should not happen with your schema), default to False
    intention_high_risk = pd.Series(False, index=df_current.index)
    intention_low_risk = pd.Series(False, index=df_current.index)

# 4.2 Additional INTENTION_USE-based risk (fallback if you want to widen coverage)
# NOTE: we don't know the exact semantics of numeric codes for INTENTION_USE,
# so we only use it as a secondary hint and keep the mapping conservative.

if "INTENTION_USE" in df_current.columns:
    # Example: consider larger numeric codes as "problematic / no intention / undecided"
    high_risk_INTENTION_USE_values = [
        "4", "5", "7",   # codes seen in your sample that might correspond to "no intention"/"undecided"
        "No intention",
        "Intends to stop",
        "Undecided",
    ]
    low_risk_INTENTION_USE_values = [
        "1", "2", "3",
        "Intends to continue",
        "Intends to use",
    ]
    intention_use_high_risk = is_in(df_current["INTENTION_USE"], high_risk_INTENTION_USE_values)
    intention_use_low_risk = is_in(df_current["INTENTION_USE"], low_risk_INTENTION_USE_values)
else:
    intention_use_high_risk = pd.Series(False, index=df_current.index)
    intention_use_low_risk = pd.Series(False, index=df_current.index)

# Combine intention signals
any_intention_high_risk = intention_high_risk | intention_use_high_risk
any_intention_low_risk = intention_low_risk | intention_use_low_risk

# 4.3 Negative history:
#     To stay error-free and general, we treat any non-empty LAST_METHOD_DISCONTINUED
#     with non-empty REASON_DISCONTINUED as a "negative history" of discontinuation.
if "LAST_METHOD_DISCONTINUED" in df_current.columns:
    last_method_nonempty = df_current["LAST_METHOD_DISCONTINUED"].astype(str).str.strip() != ""
else:
    last_method_nonempty = pd.Series(False, index=df_current.index)

if "REASON_DISCONTINUED" in df_current.columns:
    reason_nonempty = df_current["REASON_DISCONTINUED"].astype(str).str.strip() != ""
else:
    reason_nonempty = pd.Series(False, index=df_current.index)

has_negative_history = last_method_nonempty & reason_nonempty

# 4.4 Final target: HIGH_RISK_DISCONTINUE
# High risk = 1 if:
#   - any_intention_high_risk OR
#   - has_negative_history
#
# Low risk = 0 if:
#   - any_intention_low_risk AND NOT high-risk condition
#
# Others = NaN (unknown)
target = np.where(
    any_intention_high_risk | has_negative_history,
    1,
    np.where(
        any_intention_low_risk & ~(any_intention_high_risk | has_negative_history),
        0,
        np.nan
    )
)

df_current["HIGH_RISK_DISCONTINUE"] = target

print("Target value counts (including NaN):")
print(df_current["HIGH_RISK_DISCONTINUE"].value_counts(dropna=False))

# Keep only rows with defined target
df_model_d1 = df_current.dropna(subset=["HIGH_RISK_DISCONTINUE"]).copy()
df_model_d1["HIGH_RISK_DISCONTINUE"] = df_model_d1["HIGH_RISK_DISCONTINUE"].astype(int)

print("\nDesign 1 modeling sample shape:", df_model_d1.shape)
print("Target distribution (%):")
print((df_model_d1["HIGH_RISK_DISCONTINUE"].value_counts(normalize=True) * 100).round(2))

# -------------------------------------------------------------------
# 5. Define feature set (no direct label leakage)
#    We EXCLUDE:
#      - CONTRACEPTIVE_USE_AND_INTENTION
#      - INTENTION_USE
#      - HIGH_RISK_DISCONTINUE (the target itself)
# -------------------------------------------------------------------

feature_cols_demo = [
    "AGE", "REGION", "EDUC_LEVEL", "RELIGION", "ETHNICITY",
    "MARITAL_STATUS", "RESIDING_WITH_PARTNER",
    "HOUSEHOLD_HEAD_SEX", "OCCUPATION",
    "HUSBANDS_EDUC", "HUSBAND_AGE", "PARTNER_EDUC",
    "SMOKE_CIGAR"
]

feature_cols_fertility = [
    "PARITY",
    "DESIRE_FOR_MORE_CHILDREN",
    "WANT_LAST_CHILD",
    "WANT_LAST_PREGNANCY",
]

feature_cols_method = [
    "CONTRACEPTIVE_METHOD",
    "MONTH_USE_CURRENT_METHOD",
    "PATTERN_USE",
    "TOLD_ABT_SIDE_EFFECTS",
    "LAST_SOURCE_TYPE",
    "LAST_METHOD_DISCONTINUED",
    "REASON_DISCONTINUED",
    "HSBND_DESIRE_FOR_MORE_CHILDREN",
]

# Columns that directly define or heavily overlap with the target and should be excluded
leakage_cols = [
    "CONTRACEPTIVE_USE_AND_INTENTION",
    "INTENTION_USE",
    "HIGH_RISK_DISCONTINUE",
]

# Combine and keep only columns that actually exist in df_model_d1 and are not leakage
all_candidate_features = feature_cols_demo + feature_cols_fertility + feature_cols_method
feature_cols = [
    c for c in all_candidate_features
    if c in df_model_d1.columns and c not in leakage_cols
]

print("\nNumber of selected features:", len(feature_cols))
print("Selected feature columns:")
print(feature_cols)

# -------------------------------------------------------------------
# 6. Build X, y and (optionally) train-test split
# -------------------------------------------------------------------

TARGET_D1 = "HIGH_RISK_DISCONTINUE"

X = df_model_d1[feature_cols].copy()
y = df_model_d1[TARGET_D1].copy()

print("\nFinal X shape:", X.shape)
print("Final y shape:", y.shape)

# Optional: train-test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("\nTrain shape:", X_train.shape, "Test shape:", X_test.shape)
print("Train target distribution (%):")
print((y_train.value_counts(normalize=True) * 100).round(2))

"""
At this point you have:
- df_model_d1: filtered current users with binary target HIGH_RISK_DISCONTINUE
- X, y: ready for preprocessing (imputation, encoding) and modeling
You can now plug X_train, X_test, y_train, y_test into your XGBoost + Decision Tree
hybrid pipeline defined earlier.
"""

Raw data shape: (6612, 31)
Columns: ['CASEID', 'AGE', 'AGE_GRP', 'REGION', 'EDUC_LEVEL', 'RELIGION', 'ETHNICITY', 'EDUC', 'HOUSEHOLD_HEAD_SEX', 'PARITY', 'CONTRACEPTIVE_METHOD', 'CURRENT_USE_TYPE', 'LAST_SOURCE_TYPE', 'MONTH_USE_CURRENT_METHOD', 'LAST_METHOD_DISCONTINUED', 'REASON_DISCONTINUED', 'PATTERN_USE', 'INTENTION_USE', 'CONTRACEPTIVE_USE_AND_INTENTION', 'WANT_LAST_CHILD', 'WANT_LAST_PREGNANCY', 'TOLD_ABT_SIDE_EFFECTS', 'SMOKE_CIGAR', 'MARITAL_STATUS', 'RESIDING_WITH_PARTNER', 'DESIRE_FOR_MORE_CHILDREN', 'HSBND_DESIRE_FOR_MORE_CHILDREN', 'OCCUPATION', 'HUSBANDS_EDUC', 'HUSBAND_AGE', 'PARTNER_EDUC']
Filtered to current users. Shape: (3521, 31)
Target value counts (including NaN):
HIGH_RISK_DISCONTINUE
1.0    2425
0.0    1096
Name: count, dtype: int64

Design 1 modeling sample shape: (3521, 32)
Target distribution (%):
HIGH_RISK_DISCONTINUE
1    68.87
0    31.13
Name: proportion, dtype: float64

Number of selected features: 25
Selected feature columns:
['AGE', 'REGION', 'EDUC_LEVE

'\nAt this point you have:\n- df_model_d1: filtered current users with binary target HIGH_RISK_DISCONTINUE\n- X, y: ready for preprocessing (imputation, encoding) and modeling\nYou can now plug X_train, X_test, y_train, y_test into your XGBoost + Decision Tree\nhybrid pipeline defined earlier.\n'

In [5]:
import joblib
joblib.dump((X_train, X_test, y_train, y_test), '../../data/processed/discontinuation_design1_data.pkl')
joblib.dump(df_model_d1, '../../data/processed/discontinuation_design1_full_data.pkl')

['../../data/processed/discontinuation_design1_full_data.pkl']

In [6]:
"""
Hybrid model training for Design 1 (HIGH_RISK_DISCONTINUE):
- Uses X_train, X_test, y_train, y_test from the previous block.
- Builds a preprocessing pipeline (impute + one-hot).
- Trains:
    1) XGBoost classifier
    2) Decision Tree classifier
    3) Hybrid: XGBoost with Decision Tree override on low-confidence cases.
"""

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

# -------------------------------------------------------------------
# 1. Identify categorical vs numeric features
# -------------------------------------------------------------------

# X_train already defined; use its dtypes
all_features = list(X_train.columns)

# Simple rule: numeric dtypes -> numeric; others -> categorical
numeric_features = [
    col for col in all_features
    if pd.api.types.is_numeric_dtype(X_train[col])
]

categorical_features = [
    col for col in all_features
    if col not in numeric_features
]

print("Numeric features:", numeric_features)
print("Categorical features:", categorical_features)

# -------------------------------------------------------------------
# 2. Define preprocessing: imputation + encoding
# -------------------------------------------------------------------

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# -------------------------------------------------------------------
# 3. XGBoost baseline pipeline
# -------------------------------------------------------------------

xgb_clf = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    tree_method="hist",
    eval_metric="logloss",
    random_state=42,
    scale_pos_weight=None  # you can tune using class imbalance ratio
)

xgb_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", xgb_clf)
])

print("\n=== Training XGBoost baseline ===")
xgb_pipeline.fit(X_train, y_train)
y_pred_xgb = xgb_pipeline.predict(X_test)

print("\n=== XGBoost performance (HIGH_RISK_DISCONTINUE) ===")
print(classification_report(y_test, y_pred_xgb, digits=3))
print("Confusion matrix (XGBoost):")
print(confusion_matrix(y_test, y_pred_xgb))

# -------------------------------------------------------------------
# 4. Decision Tree baseline pipeline
# -------------------------------------------------------------------

dt_clf = DecisionTreeClassifier(
    max_depth=5,
    min_samples_leaf=50,
    random_state=42,
    class_weight="balanced"  # helps with imbalance
)

dt_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", dt_clf)
])

print("\n=== Training Decision Tree baseline ===")
dt_pipeline.fit(X_train, y_train)
y_pred_dt = dt_pipeline.predict(X_test)

print("\n=== Decision Tree performance (HIGH_RISK_DISCONTINUE) ===")
print(classification_report(y_test, y_pred_dt, digits=3))
print("Confusion matrix (Decision Tree):")
print(confusion_matrix(y_test, y_pred_dt))

# -------------------------------------------------------------------
# 5. Hybrid model: XGBoost + Decision Tree override
#    Strategy:
#      - Use XGBoost as main model (probabilistic).
#      - When XGBoost's predicted probability is low-confidence,
#        override with Decision Tree prediction.
# -------------------------------------------------------------------

# For binary classification, XGBClassifier already returns proba for positive class
# through predict_proba(...)[..., 1].

print("\n=== Training hybrid components ===")

# Reuse the same preprocessor, but to be explicit we build new pipelines
# (you can reuse xgb_pipeline and dt_pipeline if you prefer).

xgb_hybrid = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    tree_method="hist",
    eval_metric="logloss",
    random_state=42,
)

xgb_pipeline_hybrid = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", xgb_hybrid)
])

dt_hybrid = DecisionTreeClassifier(
    max_depth=5,
    min_samples_leaf=50,
    random_state=42,
    class_weight="balanced"
)

dt_pipeline_hybrid = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", dt_hybrid)
])

# Fit both on the same train set
xgb_pipeline_hybrid.fit(X_train, y_train)
dt_pipeline_hybrid.fit(X_train, y_train)

# XGBoost probabilities and baseline prediction
proba_xgb = xgb_pipeline_hybrid.predict_proba(X_test)[:, 1]  # P(y=1)
pred_xgb = (proba_xgb >= 0.5).astype(int)

# Decision Tree prediction
pred_dt = dt_pipeline_hybrid.predict(X_test)

# Hybrid rule:
# - If XGBoost confidence (distance to 0.5) is low, trust DT instead.
#   For binary, we can say "confidence" = |p - 0.5|.
#   If confidence < CONF_MARGIN, we override by DT.

CONF_MARGIN = 0.1  # you can tune; smaller = fewer overrides
confidence_xgb = np.abs(proba_xgb - 0.5)

use_dt = confidence_xgb < CONF_MARGIN
hybrid_pred = np.where(use_dt, pred_dt, pred_xgb)

print("\n=== Hybrid (XGBoost + Decision Tree) performance ===")
print(classification_report(y_test, hybrid_pred, digits=3))
print("Confusion matrix (Hybrid):")
print(confusion_matrix(y_test, hybrid_pred))

# Optional: inspect how often the DT override is used
override_rate = use_dt.mean() * 100
print(f"\nFraction of test cases overridden by Decision Tree: {override_rate:.2f}%")

"""
At this point you have:
- xgb_pipeline, dt_pipeline: standalone models for HIGH_RISK_DISCONTINUE
- xgb_pipeline_hybrid, dt_pipeline_hybrid, and hybrid_pred:
  a hybrid system where Decision Tree helps when XGBoost is uncertain.

You can now:
- Tune XGBoost hyperparameters (n_estimators, max_depth, learning_rate, scale_pos_weight).
- Tune DecisionTree (max_depth, min_samples_leaf).
- Tune CONF_MARGIN to trade off:
    * reliance on XGBoost vs DT
    * precision vs recall for the high-risk class.
- Save the pipelines with joblib for deployment.
"""

Numeric features: ['AGE', 'PARITY']
Categorical features: ['REGION', 'EDUC_LEVEL', 'RELIGION', 'ETHNICITY', 'MARITAL_STATUS', 'RESIDING_WITH_PARTNER', 'HOUSEHOLD_HEAD_SEX', 'OCCUPATION', 'HUSBANDS_EDUC', 'HUSBAND_AGE', 'PARTNER_EDUC', 'SMOKE_CIGAR', 'DESIRE_FOR_MORE_CHILDREN', 'WANT_LAST_CHILD', 'WANT_LAST_PREGNANCY', 'CONTRACEPTIVE_METHOD', 'MONTH_USE_CURRENT_METHOD', 'PATTERN_USE', 'TOLD_ABT_SIDE_EFFECTS', 'LAST_SOURCE_TYPE', 'LAST_METHOD_DISCONTINUED', 'REASON_DISCONTINUED', 'HSBND_DESIRE_FOR_MORE_CHILDREN']

=== Training XGBoost baseline ===

=== XGBoost performance (HIGH_RISK_DISCONTINUE) ===
              precision    recall  f1-score   support

           0      1.000     1.000     1.000       219
           1      1.000     1.000     1.000       486

    accuracy                          1.000       705
   macro avg      1.000     1.000     1.000       705
weighted avg      1.000     1.000     1.000       705

Confusion matrix (XGBoost):
[[219   0]
 [  0 486]]

=== Training Dec

'\nAt this point you have:\n- xgb_pipeline, dt_pipeline: standalone models for HIGH_RISK_DISCONTINUE\n- xgb_pipeline_hybrid, dt_pipeline_hybrid, and hybrid_pred:\n  a hybrid system where Decision Tree helps when XGBoost is uncertain.\n\nYou can now:\n- Tune XGBoost hyperparameters (n_estimators, max_depth, learning_rate, scale_pos_weight).\n- Tune DecisionTree (max_depth, min_samples_leaf).\n- Tune CONF_MARGIN to trade off:\n    * reliance on XGBoost vs DT\n    * precision vs recall for the high-risk class.\n- Save the pipelines with joblib for deployment.\n'

In [8]:
joblib.dump(xgb_pipeline_hybrid, '../models/discontinuation_design1_xgb_hybrid_model_d1.pkl')
joblib.dump(dt_pipeline_hybrid, '../models/discontinuation_design1_dt_hybrid_model_d1.pkl')
joblib.dump(xgb_pipeline, '../models/discontinuation_design1_xgb_model_d1.pkl')
joblib.dump(dt_pipeline, '../models/discontinuation_design1_dt_model_d1.pkl')

['../models/discontinuation_design1_dt_model_d1.pkl']